In [3]:
import tensorflow as tf
from tensorflow.keras import models,layers


In [4]:
IMAGE_SIZE=256
BATCH_SIZE=32
channels=3
EPOCHS=20

In [5]:
tf_dataset=tf.keras.preprocessing.image_dataset_from_directory("PlantVillage",)


Found 2152 files belonging to 3 classes.


In [6]:
len(tf_dataset)

68

In [7]:
classnames=tf_dataset.class_names
print(classnames)

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']


In [8]:
n_classes=len(classnames)

In [9]:
for image,label in tf_dataset.take(1):
    print(image[0])
    print(label)

tf.Tensor(
[[[ 15.  15.  17.]
  [ 14.  14.  16.]
  [ 14.  14.  16.]
  ...
  [153. 150. 157.]
  [138. 135. 142.]
  [124. 121. 128.]]

 [[ 15.  15.  17.]
  [ 14.  14.  16.]
  [ 14.  14.  16.]
  ...
  [142. 139. 146.]
  [150. 147. 154.]
  [160. 157. 164.]]

 [[ 15.  15.  17.]
  [ 14.  14.  16.]
  [ 14.  14.  16.]
  ...
  [150. 147. 154.]
  [155. 152. 159.]
  [159. 156. 163.]]

 ...

 [[178. 175. 184.]
  [183. 180. 189.]
  [186. 183. 192.]
  ...
  [182. 179. 186.]
  [187. 184. 191.]
  [191. 188. 195.]]

 [[180. 177. 186.]
  [184. 181. 190.]
  [186. 183. 192.]
  ...
  [180. 177. 184.]
  [181. 178. 185.]
  [183. 180. 187.]]

 [[179. 176. 185.]
  [183. 180. 189.]
  [184. 181. 190.]
  ...
  [185. 182. 189.]
  [182. 179. 186.]
  [181. 178. 185.]]], shape=(256, 256, 3), dtype=float32)
tf.Tensor([0 1 1 0 0 0 1 0 0 1 1 2 1 0 0 2 0 0 0 1 2 1 0 0 1 0 0 0 1 0 1 0], shape=(32,), dtype=int32)


In [10]:
def get_dataset_partitions_tf(tf_dataset,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=1000):
    ds_size=len(tf_dataset)
    if shuffle:
       tf_dataset= tf_dataset.shuffle(shuffle_size,seed=12)
    train_size=int(train_split*ds_size)
    train_ds=tf_dataset.take(train_size)
    val_size=int(val_split*ds_size)
    val_ds=tf_dataset.skip(train_size).take(val_size)
    test_ds=tf_dataset.skip(train_size).skip(val_size)
    return train_ds,val_ds,test_ds

In [11]:
train_ds,val_ds,test_ds=get_dataset_partitions_tf(tf_dataset)

In [12]:
print(train_ds)

<_TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [13]:
len(train_ds)

54

In [14]:
68*0.8

54.400000000000006

In [15]:
len(val_ds)

6

In [16]:
len(test_ds)

8

In [17]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [18]:
resize_and_rescale=tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

In [19]:
data_augmentation=tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

In [20]:
input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,channels)
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
     layers.MaxPooling2D((2,2)),
     layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
     layers.MaxPooling2D((2,2)),
     layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
     layers.MaxPooling2D((2,2)),
     layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
     layers.MaxPooling2D((2,2)),
     layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
     layers.MaxPooling2D((2,2)),
      layers.Flatten(),
      layers.Dense(64,activation='relu'),
      layers.Dense(n_classes,activation='softmax'),
])
model.build(input_shape=input_shape)

C:\Users\Acer\Downloads\Potato_disease\training\me_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [22]:
model.fit(train_ds,epochs=EPOCHS,batch_size=BATCH_SIZE,verbose=1,validation_data=val_ds)

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.5034 - loss: 0.9108 - val_accuracy: 0.6979 - val_loss: 0.7034
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 112s 2s/step - accuracy: 0.7382 - loss: 0.6675 - val_accuracy: 0.8542 - val_loss: 0.3506
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.8189 - loss: 0.4222 - val_accuracy: 0.8906 - val_loss: 0.2424
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step - accuracy: 0.8902 - loss: 0.2911 - val_accuracy: 0.9115 - val_loss: 0.2889
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 103s 2s/step - accuracy: 0.8856 - loss: 0.2720 - val_accuracy: 0.9219 - val_loss: 0.2168
Epoch 6/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step - accuracy: 0.9047 - loss: 0.2467 - val_accuracy: 0.9427 - val_loss: 0.1916
Epoch 7/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - accuracy: 0.9142 - loss: 0.1964 - val_accuracy: 0.8698 - val_loss: 0.3702
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step - accuracy: 0.9180 - loss: 0.2017 - val_accuracy: 0.9010 - va

In [23]:
scores=model.evaluate(test_ds)

8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 417ms/step - accuracy: 0.9507 - loss: 0.1140


In [27]:
scores

[0.10998126864433289, 0.95703125]

In [28]:
model_version=3
model.export(f"../Saved_models/{model_version}")

INFO:tensorflow:Assets written to: ../Saved_models/3\assets


INFO:tensorflow:Assets written to: ../Saved_models/3\assets


Saved artifact at '../Saved_models/3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2776451198544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451199600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451660992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451657824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451659936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451660816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451659232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451748032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451747504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451744512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2776451659584: TensorSpec(shape=(

In [29]:
model.save(f"../Saved_models/f.h5")

In [34]:
model.save(f"../Saved_models/my_model.keras")